In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/Emotion_recognition/Train'


In [ ]:
if tf.test.gpu_device_name():
    print('GPU device found:', tf.test.gpu_device_name())
else:
    print('No GPU device found')

GPU device found: /device:GPU:0


In [ ]:
!nvidia-smi

Thu Feb 15 12:42:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0              29W /  70W |  14079MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Load libraries

In [ ]:
from keras.models import Sequential
from keras.utils import to_categorical

from keras.layers import Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Dense, Dropout

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Functions

In [ ]:
def read_images(training_dir, categories, img_size=224):
    images = []
    img_size = img_size

    for cat in categories:
        path = os.path.join(training_dir, cat)
        class_num = categories.index(cat)
        print(cat, class_num)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img))
            new_img = cv2.resize(img_array, (img_size, img_size))
            gray_image = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)
            images.append([gray_image, class_num])
    return images

In [ ]:
def split_images_into_X_y(images, flatten = False, img_size =224):

    img_size = img_size
    X = []
    y = []

    for pict, label in images:
        X.append(pict)
        y.append(label)

    X = np.array(X)
    y = np.array(y)

    X = X / 255.0

    if flatten:
        X_flatten = X.reshape((-1, img_size*img_size))
        return X_flatten, y
    else:
        return X, y

# Train, test, validation split


In [ ]:
categories = ['0', '1', '2', '3', '4', '5', '6']

In [ ]:
images = read_images(file_path, categories, img_size=224)

0 0
1 1
2 2
3 3
4 4
5 5
6 6


In [ ]:
X, y = split_images_into_X_y(images, flatten = False, img_size =224)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Model 1

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 1)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(7, activation='softmax'))

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    X_train,
    to_categorical(y_train),
    epochs=30,
    batch_size=64,
    validation_data=(X_val, to_categorical(y_val))
)

Epoch 1/30
38/38 [==============================] - 48s 789ms/step - loss: 2.5923 - accuracy: 0.1482 - val_loss: 1.9460 - val_accuracy: 0.0967
Epoch 2/30
38/38 [==============================] - 17s 453ms/step - loss: 1.9400 - accuracy: 0.1834 - val_loss: 1.9335 - val_accuracy: 0.1673
Epoch 3/30
38/38 [==============================] - 17s 451ms/step - loss: 1.8743 - accuracy: 0.2425 - val_loss: 1.8683 - val_accuracy: 0.2602
Epoch 4/30
38/38 [==============================] - 18s 461ms/step - loss: 1.7603 - accuracy: 0.3121 - val_loss: 1.7893 - val_accuracy: 0.2565
Epoch 5/30
38/38 [==============================] - 18s 463ms/step - loss: 1.6132 - accuracy: 0.3961 - val_loss: 1.8037 - val_accuracy: 0.2639
Epoch 6/30
38/38 [==============================] - 17s 457ms/step - loss: 1.3986 - accuracy: 0.4776 - val_loss: 1.8461 - val_accuracy: 0.3011
Epoch 7/30
38/38 [==============================] - 18s 466ms/step - loss: 1.0912 - accuracy: 0.6167 - val_loss: 1.9774 - val_accuracy: 0.2714

In [ ]:
model.save("/content/drive/MyDrive/Emotion_recognition/model_CNN.h5")

NameError: name 'model' is not defined

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, to_categorical(y_test))

21/21 [==============================] - 3s 38ms/step - loss: 4.6447 - accuracy: 0.3170


# Model 2


In [1]:
model_2 = Sequential()

model_2.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 1)))

model_2.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model_2.add(MaxPooling2D((2, 2)))
model_2.add(Dropout(0.25))

model_2.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_2.add(MaxPooling2D((2, 2)))
#model_2.add(Dropout(0.1))

model_2.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_2.add(MaxPooling2D((2, 2)))
model_2.add(Dropout(0.25))

model_2.add(Flatten())
model_2.add(Dense(1024, activation='relu'))
model_2.add(Dropout(0.5))

model_2.add(Dense(7, activation='softmax'))

NameError: name 'Sequential' is not defined

In [ ]:
model_2.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(patience=10, restore_best_weights=True)

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler

def lr_schedule(epoch):
    if epoch < 10:
        return 0.001
    elif epoch < 20:
        return 0.0005
    else:
        return 0.0001

lr_scheduler = LearningRateScheduler(lr_schedule)

In [ ]:
history = model_2.fit(
    X_train,
    to_categorical(y_train),
    epochs=50,
    batch_size=64,
    steps_per_epoch=len(X_train) // 64,
    validation_data=(X_val, to_categorical(y_val)),
    #callbacks=[lr_scheduler]
)

In [ ]:
test_loss, test_accuracy = model_2.evaluate(X_test, to_categorical(y_test))

# Model 3

In [ ]:
model_3 = Sequential()

model_3.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 1)))
model_3.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_3.add(MaxPooling2D((2, 2)))
model_3.add(Dropout(0.2))

model_3.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_3.add(MaxPooling2D((2, 2)))
model_3.add(Dropout(0.1))

model_3.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model_3.add(MaxPooling2D((2, 2)))
model_3.add(Dropout(0.1))

model_3.add(Flatten())
model_3.add(Dense(512, activation='relu'))
model_3.add(Dropout(0.2))

model_3.add(Dense(7, activation='softmax'))


model_3.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model_3.fit(
    X_train,
    to_categorical(y_train),
    epochs=50,
    batch_size=32,
    #steps_per_epoch=len(X_train) // 64,
    validation_data=(X_val, to_categorical(y_val)),
    #callbacks=[lr_scheduler]
)

Epoch 1/50
76/76 [==============================] - 50s 514ms/step - loss: 2.2782 - accuracy: 0.1494 - val_loss: 1.9549 - val_accuracy: 0.1487
Epoch 2/50
76/76 [==============================] - 33s 428ms/step - loss: 1.9454 - accuracy: 0.1585 - val_loss: 1.9424 - val_accuracy: 0.1933
Epoch 3/50
76/76 [==============================] - 33s 428ms/step - loss: 1.9297 - accuracy: 0.2099 - val_loss: 1.8744 - val_accuracy: 0.2193
Epoch 4/50
76/76 [==============================] - 33s 428ms/step - loss: 1.7843 - accuracy: 0.2972 - val_loss: 1.8988 - val_accuracy: 0.2305
Epoch 5/50
76/76 [==============================] - 32s 428ms/step - loss: 1.6118 - accuracy: 0.3841 - val_loss: 1.9403 - val_accuracy: 0.1896
Epoch 6/50
76/76 [==============================] - 32s 427ms/step - loss: 1.3482 - accuracy: 0.4950 - val_loss: 2.2680 - val_accuracy: 0.2565
Epoch 7/50
76/76 [==============================] - 33s 428ms/step - loss: 0.9193 - accuracy: 0.6772 - val_loss: 2.7310 - val_accuracy: 0.2528

In [ ]:
test_loss, test_accuracy = model_3.evaluate(X_test, to_categorical(y_test))

21/21 [==============================] - 2s 76ms/step - loss: 3.7890 - accuracy: 0.2798


# Model 4 - Batch Normalization

In [ ]:
model_4 = Sequential()

model_4.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 1)))
model_4.add(BatchNormalization())
model_4.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Dropout(0.2))

model_4.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_4.add(BatchNormalization())
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Dropout(0.1))

model_4.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model_4.add(MaxPooling2D((2, 2)))
model_4.add(Dropout(0.1))

model_4.add(Flatten())
model_4.add(Dense(512, activation='relu'))
model_4.add(BatchNormalization())
model_4.add(Dropout(0.2))

model_4.add(Dense(7, activation='softmax'))

model_4.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
history = model_4.fit(
    X_train,
    to_categorical(y_train),
    epochs=50,
    batch_size=32,
    validation_data=(X_val, to_categorical(y_val)),
)

Epoch 1/50
76/76 [==============================] - 47s 570ms/step - loss: 2.0723 - accuracy: 0.2243 - val_loss: 2.4985 - val_accuracy: 0.1487
Epoch 2/50
76/76 [==============================] - 43s 564ms/step - loss: 1.8248 - accuracy: 0.3059 - val_loss: 3.5424 - val_accuracy: 0.1264
Epoch 3/50
76/76 [==============================] - 43s 560ms/step - loss: 1.6785 - accuracy: 0.3783 - val_loss: 10.0753 - val_accuracy: 0.1450
Epoch 4/50
76/76 [==============================] - 43s 562ms/step - loss: 1.4961 - accuracy: 0.4557 - val_loss: 2.4201 - val_accuracy: 0.2119
Epoch 5/50
76/76 [==============================] - 43s 569ms/step - loss: 1.2413 - accuracy: 0.5563 - val_loss: 2.3113 - val_accuracy: 0.1338
Epoch 6/50
76/76 [==============================] - 43s 562ms/step - loss: 0.9456 - accuracy: 0.6594 - val_loss: 1.9604 - val_accuracy: 0.2714
Epoch 7/50
76/76 [==============================] - 43s 563ms/step - loss: 0.7128 - accuracy: 0.7570 - val_loss: 2.1120 - val_accuracy: 0.252

KeyboardInterrupt: 

# Model 5 - padding

In [ ]:
model_5 = Sequential()

model_5.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(224, 224, 1)))
model_5.add(BatchNormalization())
model_5.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(Dropout(0.2))

model_5.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
model_5.add(BatchNormalization())
model_5.add(MaxPooling2D((2, 2)))
model_5.add(Dropout(0.1))

model_5.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(Dropout(0.1))

model_5.add(Flatten())
model_5.add(Dense(512, activation='relu'))
model_5.add(BatchNormalization())
model_5.add(Dropout(0.2))

model_5.add(Dense(7, activation='softmax'))

model_5.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

NameError: name 'Sequential' is not defined

In [ ]:
history = model_5.fit(
    X_train,
    to_categorical(y_train),
    epochs=30,
    batch_size=16,
    validation_data=(X_val, to_categorical(y_val)),
)

Epoch 1/30


ResourceExhaustedError: Graph execution error:

Detected at node Adam/StatefulPartitionedCall_12 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local

# Model 6 - Data augmentation

In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Model
model_6 = Sequential()

model_6.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 1)))
model_6.add(BatchNormalization())
model_6.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_6.add(MaxPooling2D((2, 2)))
model_6.add(Dropout(0.2))

model_6.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_6.add(BatchNormalization())
model_6.add(MaxPooling2D((2, 2)))
model_6.add(Dropout(0.1))

model_6.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model_6.add(MaxPooling2D((2, 2)))
model_6.add(Dropout(0.1))

model_6.add(Flatten())
model_6.add(Dense(512, activation='relu'))
model_6.add(BatchNormalization())
model_6.add(Dropout(0.2))

model_6.add(Dense(7, activation='softmax'))

# Compile Model
model_6.compile(optimizer=Adam(),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

# Display Model Summary
# model_6.summary()


In [ ]:
X_train_reshaped = X_train.reshape((-1, 224, 224, 1))
X_val_reshaped = X_val.reshape((-1, 224, 224, 1))

In [ ]:
# Fit Model with Data Augmentation
batch_size = 16
epochs = 20

history = model_6.fit(
    datagen.flow(X_train_reshaped, to_categorical(y_train), batch_size=batch_size),
    epochs=epochs,
    validation_data=(X_val_reshaped, to_categorical(y_val))
)

Epoch 1/20
151/151 [==============================] - ETA: 0s - loss: 2.2525 - accuracy: 0.1482

ResourceExhaustedError: Graph execution error:

Detected at node sequential_8/conv2d_33/Relu defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib

# pre-trained model

In [ ]:
from tensorflow.keras.applications import  MobileNetV2